# GitHub on Hacker News trends analysis

In [2]:
from __future__ import print_function
import pandas as pd

## Input parameters

In [17]:
project_id = "viant-general-billing"
process_date = "20181201"

## Exploratory Data Analysis

### Github activity data

In [13]:
query = """
SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}` 
GROUP BY 1
ORDER BY 1
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  type,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181225` 
GROUP BY 1
ORDER BY 1



,type,cnt
0,CommitCommentEvent,1115
1,CreateEvent,106915
2,DeleteEvent,16723
3,ForkEvent,24961
4,GollumEvent,3250
5,IssueCommentEvent,34831
6,IssuesEvent,21214
7,MemberEvent,1786
8,PublicEvent,707
9,PullRequestEvent,34166


### Top 10 repos with the most comments in their issues

In [18]:
query = """
SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201`
WHERE type IN ( 'IssueCommentEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,cnt
0,google-test/signcla-probe-repo,327
1,Azure/azure-rest-api-specs,287
2,kubernetes/kubernetes,227
3,rust-lang/rust,207
4,apache/spark,204
5,freeCodeCamp/freeCodeCamp,196
6,everypolitician/everypolitician-data,192
7,TeamNewPipe/NewPipe,158
8,openshift/origin,140
9,NixOS/nixpkgs,126


### Top 10 repos by stars and fork event

In [19]:
query = """
SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.{0}`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  repo.name,
  SUM(IF(type='WatchEvent', 1, NULL)) AS stars,
  SUM(IF(type='ForkEvent',  1, NULL)) AS forks,
  COUNT(*) AS cnt
FROM `githubarchive.day.20181201`
WHERE type IN ('WatchEvent','ForkEvent')
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10



,name,stars,forks,cnt
0,BcRikko/NES.css,386,35,421
1,leisurelicht/wtfpython-cn,241,31,272
2,satwikkansal/wtfpython,190,30,220
3,cssanimation/css-animation-101,178,5,183
4,firecracker-microvm/firecracker,150,13,163
5,crazyandcoder/kindle_free_books,132,31,163
6,withspectrum/spectrum,132,9,141
7,afshinea/stanford-cs-230-deep-learning,120,17,137
8,algorithm-visualizer/algorithm-visualizer,119,15,134
9,olifolkerd/tabulator,114,3,117


## Hacker News data

### Top domains shared in Hacker News

In [28]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2018-12-01"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2018-12-01"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10



,domain,cnt
0,github.com,30
1,medium.com,27
2,www.youtube.com,24
3,www.nytimes.com,19
4,venturebeat.com,10
5,www.reddit.com,7
6,www.theguardian.com,6
7,en.wikipedia.org,6
8,arstechnica.com,6
9,www.wired.com,5


### What domains have the best chance of getting more than 40 upvotes?

In [32]:
query = """
SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2018-12-01"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10
""".format(process_date)

print (query)

df = pd.read_gbq(query, project_id=project_id, dialect='standard')
df.head(20)


SELECT 
  REGEXP_EXTRACT(url, '//([^/]*)/?') as domain,
  COUNTIF(score>40) as score_gt_40,
  COUNT(*) AS cnt
FROM `bigquery-public-data.hacker_news.full`
WHERE url!='' 
AND EXTRACT(DATE FROM timestamp)="2018-12-01"
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 10



,domain,score_gt_40,cnt
0,github.com,4,30
1,www.nytimes.com,4,19
2,medium.com,3,27
3,www.wsj.com,2,4
4,www.theatlantic.com,2,5
5,www.youtube.com,2,24
6,www.sbs.com.au,1,1
7,martinfowler.com,1,1
8,www.accc.gov.au,1,1
9,angel.co,1,1
